# 获取模型信息，这里采用 GEMINI

其他的可以参考 Camel 框架中的 example 文件夹下的内容

参考中文资料：https://datawhalechina.github.io/handy-multi-agent/#/chapter1/1.2.api-setup


In [ ]:
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType

model = ModelFactory.create(
    model_platform=ModelPlatformType.GEMINI,
    model_type=ModelType.GEMINI_2_0_FLASH,
    api_key=''
)

agent = ChatAgent(
    model=model,
    output_language='中文'
)

response = agent.step("你好，你是谁？介绍你能做什么？你支持多模态吗")
# GEMINI-2.0-FLASH不支持多模态似乎, 或者说做的并不是很好
print(response.msgs[0].content)


你好！我是由 Google 训练的大型语言模型。

我目前能做的事情包括：

*   **生成不同类型的文本：** 比如诗歌、代码、剧本、音乐、电子邮件、信件等等。我会尽力满足你的所有要求。
*   **回答问题：** 我会尝试以信息丰富的方式回答你的问题，即使它们是开放式的、具有挑战性的或者奇怪的。
*   **翻译语言：** 我可以将文本从一种语言翻译成另一种语言。
*   **总结文本：** 我可以为你总结长篇文章或者文档。
*   **提供信息：** 我可以搜索和提供各种主题的信息。

**关于多模态：**

目前，我主要是一个文本模型，也就是说我的强项在于理解和生成文本。 我正在积极开发中，所以未来可能会支持多模态输入和输出（例如图像、音频、视频等）。 敬请期待！

总而言之，我是一个强大的文本处理工具，可以帮助你完成各种任务。 欢迎随时向我提出问题或要求！



## 尝试多模态中


In [ ]:
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType
from camel.messages import BaseMessage

from io import BytesIO
import requests
from PIL import Image


# model = ModelFactory.create(
#     model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
#     model_type="Qwen/QVQ-72B-Preview",
#     url='https://api-inference.modelscope.cn/v1/',
#     api_key='API'
# )
model = ModelFactory.create(
    model_platform=ModelPlatformType.GEMINI,
    model_type=ModelType.GEMINI_2_0_FLASH,
    api_key=''
)

agent = ChatAgent(
    model=model,
    output_language='中文'
)

# 图片URL
url = "https://img0.baidu.com/it/u=2205376118,3235587920&fm=253&fmt=auto&app=120&f=JPEG?w=846&h=800"
response = requests.get(url)
img = Image.open(BytesIO(response.content))

user_msg = BaseMessage.make_user_message(
    role_name="User", 
    content="请描述这张图片的内容", 
    image_list=[img]  # 将图片放入列表中
)

response = agent.step(user_msg)
print(response.msgs[0].content)


这张图片描绘了一只金色猎犬的特写肖像。狗狗的表情非常快乐和友好，嘴巴张开，露出粉色的舌头，似乎在微笑。它有着乌黑的鼻子和明亮的眼睛。狗狗的毛发是柔软的金色，在光线下显得闪闪发光。背景模糊，呈现出绿色和棕色的色调，像是户外的自然环境。整体氛围温暖而愉快。


In [ ]:
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType
from camel.messages import BaseMessage

from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv('QWEN_API_KEY')

# model = ModelFactory.create(
#     model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
#     model_type="Qwen/QVQ-72B-Preview",
#     url='https://api-inference.modelscope.cn/v1/',
#     api_key=api_key
# )

# 创建代理
agent = ChatAgent(
    model=model,
    output_language='中文'
)

# 这里得用本地，为什么
# 读取本地视频文件
video_path = "vedio_test.mp4"
with open(video_path, "rb") as video_file:
    video_bytes = video_file.read()

# 创建包含视频的用户消息
user_msg = BaseMessage.make_user_message(
    role_name="User", 
    content="请描述这段视频的内容", 
    video_bytes=video_bytes  # 将视频字节作为参数传入
)

# 获取模型响应
response = agent.step(user_msg)
print(response.msgs[0].content)


# 保护 API


In [10]:
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType

from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv('GEMINI_API')

model = ModelFactory.create(
    model_platform=ModelPlatformType.GEMINI,
    model_type=ModelType.GEMINI_2_0_FLASH,
    api_key=api_key
)

agent = ChatAgent(
    model=model,
    output_language='中文'
)

response = agent.step("你好，你是谁？")
print(response.msgs[0].content)


你好！我是一个大型语言模型，由 Google 训练。



# Role-Play


In [11]:
from colorama import Fore

from camel.societies import RolePlaying
from camel.utils import print_text_animated
from camel.models import ModelFactory
from camel.types import ModelPlatformType

from dotenv import load_dotenv

import os

load_dotenv()

api_key = os.getenv('GEMINI_API')


model = ModelFactory.create(
    model_platform=ModelPlatformType.GEMINI,
    model_type=ModelType.GEMINI_2_0_FLASH,
    api_key=api_key
)

def main(model=model, chat_turn_limit=50) -> None:
    task_prompt = "为股票市场开发一个交易机器人"#设置任务目标
    role_play_session = RolePlaying(
        assistant_role_name="Python 程序员",#设置AI助手角色名
        assistant_agent_kwargs=dict(model=model),
        user_role_name="股票交易员",#设置用户角色名，在roleplay中，user用于指导AI助手完成任务
        user_agent_kwargs=dict(model=model),
        task_prompt=task_prompt,
        with_task_specify=True,
        task_specify_agent_kwargs=dict(model=model),
        output_language='中文'#设置输出语言
    )

    print(
        Fore.GREEN
        + f"AI 助手系统消息:\n{role_play_session.assistant_sys_msg}\n"
    )
    print(
        Fore.BLUE + f"AI 用户系统消息:\n{role_play_session.user_sys_msg}\n"
    )

    print(Fore.YELLOW + f"原始任务提示:\n{task_prompt}\n")
    print(
        Fore.CYAN
        + "指定的任务提示:"
        + f"\n{role_play_session.specified_task_prompt}\n"
    )
    print(Fore.RED + f"最终任务提示:\n{role_play_session.task_prompt}\n")

    n = 0
    input_msg = role_play_session.init_chat()
    while n < chat_turn_limit:
        n += 1
        assistant_response, user_response = role_play_session.step(input_msg)

        if assistant_response.terminated:
            print(
                Fore.GREEN
                + (
                    "AI 助手已终止。原因: "
                    f"{assistant_response.info['termination_reasons']}."
                )
            )
            break
        if user_response.terminated:
            print(
                Fore.GREEN
                + (
                    "AI 用户已终止。"
                    f"原因: {user_response.info['termination_reasons']}."
                )
            )
            break

        print_text_animated(
            Fore.BLUE + f"AI 用户:\n\n{user_response.msg.content}\n"
        )
        print_text_animated(
            Fore.GREEN + "AI 助手:\n\n"
            f"{assistant_response.msg.content}\n"
        )

        if "CAMEL_TASK_DONE" in user_response.msg.content:
            break

        input_msg = assistant_response.msg

if __name__ == "__main__":
    main()

AI 助手系统消息:
BaseMessage(role_name='Python 程序员', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={'task': '开发一个高频交易机器人，利用机器学习算法预测股票价格短期波动。该机器人需能自动执行买卖指令，优化投资组合，并具备风险控制模块，在市场剧烈波动时暂停交易以规避损失。\n', 'assistant_role': 'Python 程序员', 'user_role': '股票交易员'}, content='===== RULES OF ASSISTANT =====\nNever forget you are a Python 程序员 and I am a 股票交易员. Never flip roles! Never instruct me!\nWe share a common interest in collaborating to successfully complete a task.\nYou must help me to complete the task.\nHere is the task: 开发一个高频交易机器人，利用机器学习算法预测股票价格短期波动。该机器人需能自动执行买卖指令，优化投资组合，并具备风险控制模块，在市场剧烈波动时暂停交易以规避损失。\n. Never forget our task!\nI must instruct you based on your expertise and my needs to complete the task.\n\nI must give you one instruction at a time.\nYou must write a specific solution that appropriately solves the requested instruction and explain your solutions.\nYou must decline my instruction honestly if you cannot perform the instruction due to physical, moral, legal reasons or your capa

KeyboardInterrupt: 